**First Steps in Programming a Humanoid AI Robot - Summer 2023**

# **Training an Object Detector** - Google CoLab

This is a JupyterLab notebook that helps you train YOLOv3 with a custom data set. The instructions are based on [this excellent tutorial](https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects).

To make sure the training is reasonably fast, we have to use a runtime with a GPU. From the menu, select "Runtime" -> "Change run type" and make sure GPU is selected as your hardware accelerator.



---



## 0. Environment setup

Make sure you have created the folders `isp/custom_data` and `isp/custom_test` in your Google Drive.

Let's import some Python libraries that are used by our notebook.

_Hint_: You can execute this and all following code cells by clicking the run icon that appears when you move your mouse into the cell.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import glob
import os, sys
import torch
from google.colab import drive

## 1. Clone and compile YOLOv3

First, we clone Darknet - the framework used to train our YOLOv3 object detector - into CoLab.

SNU's firewall may reject the first connection attempt by git (`fatal: unable to access https://teaching.csap.snu.ac.kr...`). If that happens, simply execute the cell again.

In [ ]:
!rm -rf darknet
!git clone https://teaching.csap.snu.ac.kr/first-steps-in-programming-a-humanoid-ai-robot/isp-2023/darknet.git

Next, let's find out what GPU type our runtime provides.

In [ ]:
ARCH=""
HAS_GPU=0
HAS_HALF=0

if torch.cuda.is_available():
  gpu_name = torch.cuda.get_device_name(0)
  print(f"GPU Type: {gpu_name}")
  HAS_GPU=1
  HAS_HALF=1
  if "T4" in gpu_name:
    ARCH=" -gencode arch=compute_75,code=[sm_75,compute_75]"
  elif "V100" in gpu_name:
    ARCH=" -gencode arch=compute_70,code=[sm_70,compute_70]"
  elif "A100" in gpu_name:
    ARCH=" -gencode arch=compute_80,code=[sm_80,compute_80]"
  else:
    print("GPU type not recognized. Add your GPU type to this code here.")
    HAS_GPU=0
    HAS_HALF=0
else:
  print(f"CUDA is not available. Change the runtime type (Runtime -> Change runtime type)")

print(f"  Configuration:\n"
      f"    GPU={HAS_GPU}\n"
      f"    CUDNN={HAS_GPU}\n"
      f"    CUDNN_HALF={HAS_HALF}\n"
      f"    ARCH={ARCH}")

Next, we modify the Makefile to match our system architecture.

In [ ]:
%cd /content/darknet
!sed -i "s/GPU=[01]/GPU=$HAS_GPU/" Makefile
!sed -i "s/CUDNN=[01]/CUDNN=$HAS_GPU/" Makefile
!sed -i "s/CUDNN_HALF=[01]/CUDNN_HALF=$HAS_HALF/" Makefile
!sed -i "s/OPENCV=[01]/OPENCV=1/" Makefile
!sed -i "s/^ARCH=.*/ARCH=$ARCH/" Makefile

Let's compile Darknet. This will take 2-3 minutes. Once compilation has completed, you will see the message "Build complete."

In [ ]:
!make -j $((`nproc`*2))
!echo "Build complete."

## 2. Prepare the data set

Next, we duplicate the directory structure of our data set in Google Drive and copy the training data into CoLab.

`custom_info` contains configuration files and information about the training data. `custom_data` is where our training data and, during training, the weights will be stored.

In [ ]:
!mkdir custom_info
!mkdir custom_data

Give CoLab access to our Google Drive. You have to authorize the access every time you mount your Google Drive.


In [ ]:
drive.mount('/content/gdrive')
dataset_path = '/content/gdrive/MyDrive/isp/custom_data/'

Now, we copy the training data from our Google Drive `custom_data` folder into the same folder here in CoLab:

In [ ]:
!cp /content/gdrive/MyDrive/isp/custom_data/* custom_data/

Next, we save the class labes where YOLO expects them.

In [ ]:
!cp custom_data/classes.txt custom_info/obj.names

Finally, we create a file `train.txt` that lists all files we want to use for training. Per default, we include all files that are present in the `custom_data` folder.

* Remember that the test set must be separate from the training set. Make sure to set a few images aside in the `custom_test` folder on your Google Drive.
* Make sure that each image "<image>.jpg/jpeg/png" file is accompanied by a corresponding "<image>.txt" file that contains the object annotations in YOLO format.

In [ ]:
images = glob.glob("custom_data/*[jpg|jpeg|png]")
file = open("custom_info/train.txt", "w")
file.write("\n".join(images))
file.close()

## 3. Prepare the initial weights

At the very beginning of our training, we need to download the pre-initialized weight files for YOLOv3. These weights provide a starting point for our training. If that is the case, execute the first code cell below.

If your training was interrupted at some point in time, you can restore the most recent weight backup from your Google Drive and continue the training where you left off. In this case, execute the second code cell below.



No partially trained weigths available: download pre-initialized weights for YOLOv3.

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

Continue training: restore the most recent copy of your weights.

In [ ]:
!cp custom_data/yolov3_custom_object_last.weights darknet53.conv.74

## 4. Configure the training parameters

The next line, `classes = 2` must reflect the actual number of classes you want to train your network on. Make sure to adjust it to your needs before executing the cell.

In [ ]:
classes = 1

First, we create a configuration file describing our training data set and setting the weight backup location.

In [ ]:
!echo -e "classes = $classes\ntrain = custom_info/train.txt\nvalid = custom_info/test.txt\nnames = custom_info/obj.names\nbackup = /content/gdrive/MyDrive/isp/custom_data/" > custom_info/obj.data

Next, we set the training parameters in a custom configuration file `custom_info/yolo3_custom_object.cfg` copied from `cfg/yolov3.cfg`.

For starters, we recommend leaving all settings as they are. If the training aborts because of insufficient GPU memory (`Cuda malloc failed: Success`), reduce the number of batches from 64 to something smaller.


In [ ]:
filters= (classes + 5)*3
max_batches = classes*2000
max_batches = max(6000, max_batches)
steps_low = max_batches * 80 // 100
steps_high = max_batches * 90 // 100

!cp cfg/yolov3.cfg custom_info/yolov3_custom_object.cfg
!sed -i 's/^batch=[0-9]*/batch=64/' custom_info/yolov3_custom_object.cfg
!sed -i 's/^subdivisions=[0-9]*/subdivisions=16/' custom_info/yolov3_custom_object.cfg
!sed -i "s/^max_batches=[0-9]*/max_batches=$max_batches/" custom_info/yolov3_custom_object.cfg
!sed -i "s/^steps=[0-9,]*/steps=$steps_low,$steps_high/" custom_info/yolov3_custom_object.cfg
!sed -i 's/^width=416/width=416/' custom_info/yolov3_custom_object.cfg
!sed -i 's/^height=416/height=416/' custom_info/yolov3_custom_object.cfg
!sed -i "s/^classes=80/classes=$classes/" custom_info/yolov3_custom_object.cfg
!sed -i "s/^filters=255/filters=$filters/" custom_info/yolov3_custom_object.cfg

In [ ]:
!cat custom_info/yolov3_custom_object.cfg

## 5. Train YOLOv3

Finally, we can (re-)start our training of YOLOv3. This can take considerable time (several hours to days), depending on the number of classes and number of training images.

As training progresses, Darknet will output a time estimate when training will finish.

In [ ]:
!./darknet detector train custom_info/obj.data custom_info/yolov3_custom_object.cfg darknet53.conv.74 -dont_show

While training progresses, Darknet saves snapshots of the trained weights in our Google Drive `custom_data` folder as
```
yolov3_custom_object_1000.weights
yolov3_custom_object_2000.weights
...
```

You will also find the files
```
yolov3_custom_object_last.weights
yolov3_custom_object_final.weights
```
The former is the most recent backup of the weights and can be used as a starting point to continue the training and also to test the accuracy of the predictor so far. The latter file, `yolov3_custom_object_final.weights` is generated once training completes.

## 6. Testing our object detector

At any point in time, we can test the accuracy of our custom object detector.

First, we copy the most recent weights into CoLab

In [ ]:
!cp /content/gdrive/MyDrive/isp/custom_data/yolov3_custom_object_last.weights custom_info/yolov3_custom_object_last.weights


Next, we copy the training set into CoLab

In [ ]:
!mkdir -p custom_test/
!cp /content/gdrive/MyDrive/isp/custom_test/* custom_test/

Now, we can run YOLOv3 on a test image.

The last parameter (`custom_test/test01.jpg`) indicates the image that we run our custom object detector on. Modify as needed.

In [ ]:
!./darknet classifier predict custom_info/obj.data custom_info/yolov3_custom_object.cfg custom_info/yolov3_custom_object_last.weights custom_test/img-00002-00026.jpg


To save the output in graphical form, use the following command. It saves the predictions into a file called `predictions.jpg` that we can then view with OpenCV.


In [ ]:
!./darknet detector test custom_info/obj.data custom_info/yolov3_custom_object.cfg custom_info/yolov3_custom_object_last.weights -thresh 0.2 custom_test/img-00002-00026.jpg

In [ ]:
pred_img = cv2.imread('predictions.jpg')
pred_img = cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(15,15))
plt.imshow(pred_img)